In [5]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
from pathlib import Path
import pandas as pd
from spotifuncs import create_df_from_API, append_audio_features,create_similarity_score,get_recommendations

In [6]:
path = Path("C:/Users/ms101/OneDrive/DataScience_ML/projects/spotify_app")

In [7]:
with open(path / "client_s.txt") as f:
    content = f.readlines()
content = [x.strip() for x in content]

client_id = content[0]
client_secret = content[1]

In [8]:
with open(path / "usernames.txt") as f:
    usernames = f.readlines()
usernames = [x.strip() for x in usernames]

username1 = usernames[0]
username2 = usernames[1]

In [9]:
scope = "user-library-read user-read-recently-played user-top-read playlist-modify-public playlist-read-private playlist-read-collaborative"

In [10]:
redirect_uri = "https://developer.spotify.com/dashboard/applications/4a4e029d299a4241873db8300038bf0a"


client_credentials_manager = SpotifyClientCredentials(client_id=client_id, 
                                                      client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)


token = util.prompt_for_user_token(username1, scope, client_id, client_secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username1)

Enter the URL you were redirected to: https://developer.spotify.com/dashboard/login?code=AQAoqlmLhhknKKsPoKoU9UOtvRNgHL2xOJO5zDmtixJubVs5b7QfdiXklD0G2hkWv3JNihsAHvtIVJ5aYImGw95CItYhdAdMwmdUT1olp8V0lCa4hiKxNwjZgIC8K4BUfmTMv_FPnl_QvwLjaCMRGK8qy83VP6q_3ZcevzPAoghp8PCs9CFygt84ew4UiCe3brNDkbXU_7tI7_TYG2qFFjoVHAsomPSGtKiu08kRvxRdNkzM3gGcYU4UfkJJj1QF9QHf45CR0REh4QklYdYEa6fqCwMTCQrVcVlq1043k9r1IDRvN6SacLHqsYI5JxspdFw3wJv4N1szjYSg5IlVffwvwI50BV39khlaxWTWE0E5-6LGo6ULKkeBqdybrsekn3M8bmIj5LLIkDv7KlmL1gtTXK61fgjQDsHsuE8wI-Edng


In [12]:
#user top tracks
#current_user_top_tracks tried before
#user top artists
top_artist_long = sp.current_user_top_artists(limit = 50, time_range = "long_term")
top_artist_short = sp.current_user_top_artists(limit = 50, time_range = "short_term")
#user recent tracks
#still in BETA phase no spotipy function (workaround??)

#user saved tracks
user_saved_tracks = sp.current_user_saved_tracks(limit = 50)


In [17]:
top_artist_long

{'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6wWVKhxIU2cEi0K81v7HvP'},
   'followers': {'href': None, 'total': 5230616},
   'genres': ['alternative metal',
    'german metal',
    'industrial',
    'industrial metal',
    'industrial rock',
    'neue deutsche harte',
    'nu metal'],
   'href': 'https://api.spotify.com/v1/artists/6wWVKhxIU2cEi0K81v7HvP',
   'id': '6wWVKhxIU2cEi0K81v7HvP',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/121166c58f391a367c24412d8ee29346c157105d',
     'width': 640},
    {'height': 320,
     'url': 'https://i.scdn.co/image/e53c4ffd20251515188d53057910677af3194bac',
     'width': 320},
    {'height': 160,
     'url': 'https://i.scdn.co/image/2cd3cc18d3d8926abd3387d189c190ea3190a3b6',
     'width': 160}],
   'name': 'Rammstein',
   'popularity': 79,
   'type': 'artist',
   'uri': 'spotify:artist:6wWVKhxIU2cEi0K81v7HvP'},
  {'external_urls': {'spotify': 'https://open.spotify.com/artist/2kXKa3aAFngGz2P4GjG5w2'},

In [21]:
name = []
artist_id = []
genres = []
uri = []
popularity = []

for artist in top_artist_long["items"]:
    #add name
    name.append(artist["name"])
    #add artist_id
    artist_id.append(artist["id"])
    #add genres
    genres.append(artist["genres"])
    #add popularity
    popularity.append(artist["popularity"])
    #add uri
    uri.append(artist["uri"])
    
artists_long_df = pd.DataFrame({ "name": name, 
                                "artist_id": artist_id, 
                                "genres": genres,
                                "uri": uri, 
                                "popularity": popularity})
artists_long_df

,name,artist_id,genres,uri,popularity
0,Rammstein,6wWVKhxIU2cEi0K81v7HvP,"[alternative metal, german metal, industrial, ...",spotify:artist:6wWVKhxIU2cEi0K81v7HvP,79
1,SCH,2kXKa3aAFngGz2P4GjG5w2,"[french hip hop, pop urbaine, rap francais, ra...",spotify:artist:2kXKa3aAFngGz2P4GjG5w2,77
2,G-Eazy,02kJSzxNuaWGqwubyUba0Z,"[hip hop, indie pop rap, oakland hip hop, pop ...",spotify:artist:02kJSzxNuaWGqwubyUba0Z,83
3,Eminem,7dGJo4pcD2V6oG8kP0tJRR,"[detroit hip hop, hip hop, rap]",spotify:artist:7dGJo4pcD2V6oG8kP0tJRR,92
4,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,"[canadian contemporary r&b, canadian pop, pop]",spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ,94
5,Lil Wayne,55Aa2cqylxrFIXC767Z865,"[hip hop, new orleans rap, pop rap, rap, trap]",spotify:artist:55Aa2cqylxrFIXC767Z865,89
6,Bring Me The Horizon,1Ffb6ejR6Fe5IamqA5oRUF,"[melodic metalcore, metalcore, uk metalcore]",spotify:artist:1Ffb6ejR6Fe5IamqA5oRUF,82
7,Drake,3TVXtAsR1Inumwj472S9r4,"[canadian hip hop, canadian pop, hip hop, pop ...",spotify:artist:3TVXtAsR1Inumwj472S9r4,97
8,NF,6fOMl44jA4Sp5b9PpYCkzz,"[hip hop, pop rap, rap]",spotify:artist:6fOMl44jA4Sp5b9PpYCkzz,84
9,JAY-Z,3nFkdlSjzX9mRTtwJOzDYB,"[east coast hip hop, hip hop, pop rap, rap]",spotify:artist:3nFkdlSjzX9mRTtwJOzDYB,85


In [24]:
def top_artists_from_API(api_results):
    name = []
    artist_id = []
    genres = []
    uri = []
    popularity = []
    for artist in api_results["items"]:
        #add name
        name.append(artist["name"])
        #add artist_id
        artist_id.append(artist["id"])
        #add genres
        genres.append(artist["genres"])
        #add popularity
        popularity.append(artist["popularity"])
        #add uri
        uri.append(artist["uri"])
    
    artists_df = pd.DataFrame({ "name": name, 
                                "artist_id": artist_id, 
                                "genres": genres,
                                "uri": uri, 
                                "popularity": popularity})
    return artists_df

In [37]:
def top_artists_from_API(api_results):
    df = pd.DataFrame(api_results["items"])
    cols = ["name","id","genres","popularity","uri"]
    return df[cols]

In [25]:
artists_short_df = top_artists_from_API(top_artist_short)
artists_short_df

,name,artist_id,genres,uri,popularity
0,Fynn Kliemann,2YIxQoGvBukvoC1CbJ7opS,"[german indie, german pop]",spotify:artist:2YIxQoGvBukvoC1CbJ7opS,65
1,Joyner Lucas,6C1ohJrd5VydigQtaGy5Wa,"[boston hip hop, hip hop, pop rap, rap]",spotify:artist:6C1ohJrd5VydigQtaGy5Wa,78
2,Peter Fox,6rqlONGmPuP2wJVSfliLBI,"[german hip hop, german pop, german rock]",spotify:artist:6rqlONGmPuP2wJVSfliLBI,61
3,Geegun,5W7N6u4EjCEMKj7bDyzPEC,"[russian dance, russian dance pop, russian hip...",spotify:artist:5W7N6u4EjCEMKj7bDyzPEC,57
4,Thirty Seconds To Mars,0RqtSIYZmd4fiBKVFqyIqD,"[modern rock, pop punk, post-grunge, rock]",spotify:artist:0RqtSIYZmd4fiBKVFqyIqD,70
5,SCH,2kXKa3aAFngGz2P4GjG5w2,"[french hip hop, pop urbaine, rap francais, ra...",spotify:artist:2kXKa3aAFngGz2P4GjG5w2,77
6,NF,6fOMl44jA4Sp5b9PpYCkzz,"[hip hop, pop rap, rap]",spotify:artist:6fOMl44jA4Sp5b9PpYCkzz,84
7,Die Antwoord,6urkHDoIVO1WO8vNIwcJmM,"[african rock, south african alternative, sout...",spotify:artist:6urkHDoIVO1WO8vNIwcJmM,64
8,Rammstein,6wWVKhxIU2cEi0K81v7HvP,"[alternative metal, german metal, industrial, ...",spotify:artist:6wWVKhxIU2cEi0K81v7HvP,79
9,Gidge,7aibxFH8hIlSUlXgshCgAP,"[electronica, italian electronica, microhouse]",spotify:artist:7aibxFH8hIlSUlXgshCgAP,49


In [38]:
artists_short_df = top_artists_from_API(top_artist_short)
artists_short_df

,name,id,genres,popularity,uri
0,Fynn Kliemann,2YIxQoGvBukvoC1CbJ7opS,"[german indie, german pop]",65,spotify:artist:2YIxQoGvBukvoC1CbJ7opS
1,Joyner Lucas,6C1ohJrd5VydigQtaGy5Wa,"[boston hip hop, hip hop, pop rap, rap]",78,spotify:artist:6C1ohJrd5VydigQtaGy5Wa
2,Peter Fox,6rqlONGmPuP2wJVSfliLBI,"[german hip hop, german pop, german rock]",61,spotify:artist:6rqlONGmPuP2wJVSfliLBI
3,Geegun,5W7N6u4EjCEMKj7bDyzPEC,"[russian dance, russian dance pop, russian hip...",57,spotify:artist:5W7N6u4EjCEMKj7bDyzPEC
4,Thirty Seconds To Mars,0RqtSIYZmd4fiBKVFqyIqD,"[modern rock, pop punk, post-grunge, rock]",70,spotify:artist:0RqtSIYZmd4fiBKVFqyIqD
5,SCH,2kXKa3aAFngGz2P4GjG5w2,"[french hip hop, pop urbaine, rap francais, ra...",77,spotify:artist:2kXKa3aAFngGz2P4GjG5w2
6,NF,6fOMl44jA4Sp5b9PpYCkzz,"[hip hop, pop rap, rap]",84,spotify:artist:6fOMl44jA4Sp5b9PpYCkzz
7,Die Antwoord,6urkHDoIVO1WO8vNIwcJmM,"[african rock, south african alternative, sout...",64,spotify:artist:6urkHDoIVO1WO8vNIwcJmM
8,Rammstein,6wWVKhxIU2cEi0K81v7HvP,"[alternative metal, german metal, industrial, ...",79,spotify:artist:6wWVKhxIU2cEi0K81v7HvP
9,Gidge,7aibxFH8hIlSUlXgshCgAP,"[electronica, italian electronica, microhouse]",49,spotify:artist:7aibxFH8hIlSUlXgshCgAP


In [28]:
cols = ["name","id","genres","popularity","uri"]
artists_long_df_try[cols]

,name,id,genres,popularity,uri
0,Rammstein,6wWVKhxIU2cEi0K81v7HvP,"[alternative metal, german metal, industrial, ...",79,spotify:artist:6wWVKhxIU2cEi0K81v7HvP
1,SCH,2kXKa3aAFngGz2P4GjG5w2,"[french hip hop, pop urbaine, rap francais, ra...",77,spotify:artist:2kXKa3aAFngGz2P4GjG5w2
2,G-Eazy,02kJSzxNuaWGqwubyUba0Z,"[hip hop, indie pop rap, oakland hip hop, pop ...",83,spotify:artist:02kJSzxNuaWGqwubyUba0Z
3,Eminem,7dGJo4pcD2V6oG8kP0tJRR,"[detroit hip hop, hip hop, rap]",92,spotify:artist:7dGJo4pcD2V6oG8kP0tJRR
4,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,"[canadian contemporary r&b, canadian pop, pop]",94,spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ
5,Lil Wayne,55Aa2cqylxrFIXC767Z865,"[hip hop, new orleans rap, pop rap, rap, trap]",89,spotify:artist:55Aa2cqylxrFIXC767Z865
6,Bring Me The Horizon,1Ffb6ejR6Fe5IamqA5oRUF,"[melodic metalcore, metalcore, uk metalcore]",82,spotify:artist:1Ffb6ejR6Fe5IamqA5oRUF
7,Drake,3TVXtAsR1Inumwj472S9r4,"[canadian hip hop, canadian pop, hip hop, pop ...",97,spotify:artist:3TVXtAsR1Inumwj472S9r4
8,NF,6fOMl44jA4Sp5b9PpYCkzz,"[hip hop, pop rap, rap]",84,spotify:artist:6fOMl44jA4Sp5b9PpYCkzz
9,JAY-Z,3nFkdlSjzX9mRTtwJOzDYB,"[east coast hip hop, hip hop, pop rap, rap]",85,spotify:artist:3nFkdlSjzX9mRTtwJOzDYB


In [31]:
artists_long_df_try[cols].iloc[:] == 

ValueError: Can only compare identically-labeled DataFrame objects

In [ ]:
##Playlists
#spotify features playlists + get playlist by id
featured_playlists, playlist

For later use:

current_user_saved_tracks
- Gets a list of the tracks saved in the current authorized user’s “Your Music” library

artist_top_tracks
- Get Spotify catalog information about an artist’s top 10 tracks by country. 

artist_related_artists
- Get Spotify catalog information about artists similar to an identified artist. Similarity is based on analysis of the Spotify community’s listening history.

playlist_add_items

recommendations(seed_artists=None, seed_genres=None, seed_tracks=None, limit=20, country=None, \*\*kwargs)
- Get a list of recommended tracks for one to five seeds. (at least one of seed_artists, seed_tracks and seed_genres are needed)

- Parameters:
- seed_artists - a list of artist IDs, URIs or URLs
- seed_tracks - a list of track IDs, URIs or URLs
- seed_genres - a list of genre names. Available genres for
- recommendations can be found by calling recommendation_genre_seeds

user_playlist_replace_tracks(user, playlist_id, tracks)
